In [ ]:
import pandas as pd

# 读取整个CSV文件
sl_data = pd.read_csv('./data/SLKB_rawSL.csv')

# 查看前5行
print(sl_data.head())

# # 获取特定列
# column_data = data['column_name']

# # 转换为字典列表
# data_dict = data.to_dict('records')

   Unnamed: 0     gene_pair  study_origin cell_line_origin gene_1  gene_2  \
0           1   AKT1|AMBRA1      33956155             RPE1   AKT1  AMBRA1   
1           2   AKT3|AMBRA1      33956155             RPE1   AKT3  AMBRA1   
2           3   AMBRA1|ARF6      33956155             RPE1   ARF6  AMBRA1   
3           4   AMBRA1|ATF4      33956155             RPE1   ATF4  AMBRA1   
4           5  AMBRA1|ATG10      33956155             RPE1  ATG10  AMBRA1   

  SL_or_not  SL_score  statistical_score  SL_score_cutoff  \
0    Not SL -0.010982                0.0             -1.0   
1    Not SL  2.159344                0.0             -1.0   
2    Not SL -0.564699                0.0             -1.0   
3    Not SL  0.999030                0.0             -1.0   
4    Not SL  3.916281                0.0             -1.0   

   statistical_score_cutoff  
0                       0.0  
1                       0.0  
2                       0.0  
3                       0.0  
4                   

In [11]:
# import pandas as pd

# # 读取 CSV 文件
# df = pd.read_csv('your_file.csv')
df = sl_data

# 计算每个细胞系的总数
total_counts = df['cell_line_origin'].value_counts().reset_index()
total_counts.columns = ['cell_line_origin', 'total_count']

# 计算每个细胞系中 SL 的数量
sl_counts = df[df['SL_or_not'] == 'SL']['cell_line_origin'].value_counts().reset_index()
sl_counts.columns = ['cell_line_origin', 'sl_count']

# 合并两个结果（使用左连接，确保所有细胞系都出现）
result = pd.merge(total_counts, sl_counts, on='cell_line_origin', how='left').fillna(0)
result["precent"] = result['sl_count'] / result['total_count']

print(result)

   cell_line_origin  total_count  sl_count   precent
0              K562       120919    1563.0  0.012926
1            JURKAT        75078     374.0  0.004981
2              RPE1        11911    6833.0  0.573671
3              A549         9062     682.0  0.075259
4              MEWO         6050     475.0  0.078512
5           MELJUSO         5159     551.0  0.106804
6              HELA         4933     149.0  0.030205
7            HS936T         4859     636.0  0.130891
8               GI1         4859     476.0  0.097963
9            HS944T         4859     497.0  0.102284
10        PATU8988S         4859     618.0  0.127187
11              PK1         4859     668.0  0.137477
12           MEL202         4859     688.0  0.141593
13           IPC298         4859     619.0  0.127392
14             HSC5         4859     705.0  0.145092
15             293T         2628      59.0  0.022451
16           OVCAR8         1525      56.0  0.036721
17             A375         1491      46.0  0.

In [12]:
# 读取 PKL 文件 --> dict
gfm_data = pd.read_pickle("./data/geneformer_gene_embs.pkl")

# # 查看数据（如果是 DataFrame）
# print(gfm_data[0])

In [ ]:
import pandas as pd
import numpy as np

# 假设 gfm_data 和 SLdata 已经加载
# gfm_data = {gene_name: emb, ...}
# SLdata = pd.DataFrame({"gene_1": [...], "gene_2": [...]})

# 1. 提取所有唯一基因
all_genes = pd.unique(pd.concat([sl_data["gene_1"], sl_data["gene_2"]]))
total_genes = len(all_genes)

# 4. （可选）将 emb 添加到 SLdata 中
sl_data["gene_1_emb"] = sl_data["gene_1"].map(gfm_data)  # 如果基因不存在，值为 NaN
sl_data["gene_2_emb"] = sl_data["gene_2"].map(gfm_data)

# 检查 SLdata 中 emb 的覆盖率
valid_pairs = sl_data.dropna(subset=["gene_1_emb", "gene_2_emb"])
valid_ratio = len(valid_pairs) / len(sl_data) * 100
print(f"\n有效基因对（两个基因都有 emb）的比例: {valid_ratio:.2f}%")


有效基因对（两个基因都有 emb）的比例: 77.07%


In [14]:
print(sl_data.head())

   Unnamed: 0     gene_pair  study_origin cell_line_origin gene_1  gene_2  \
0           1   AKT1|AMBRA1      33956155             RPE1   AKT1  AMBRA1   
1           2   AKT3|AMBRA1      33956155             RPE1   AKT3  AMBRA1   
2           3   AMBRA1|ARF6      33956155             RPE1   ARF6  AMBRA1   
3           4   AMBRA1|ATF4      33956155             RPE1   ATF4  AMBRA1   
4           5  AMBRA1|ATG10      33956155             RPE1  ATG10  AMBRA1   

  SL_or_not  SL_score  statistical_score  SL_score_cutoff  \
0    Not SL -0.010982                0.0             -1.0   
1    Not SL  2.159344                0.0             -1.0   
2    Not SL -0.564699                0.0             -1.0   
3    Not SL  0.999030                0.0             -1.0   
4    Not SL  3.916281                0.0             -1.0   

   statistical_score_cutoff  \
0                       0.0   
1                       0.0   
2                       0.0   
3                       0.0   
4              

In [ ]:
# geneformer emb 的覆盖率

import pandas as pd
import numpy as np


results = []

for cell_line, group in sl_data.groupby("cell_line_origin"):
    # 提取该细胞系下的所有唯一基因
    genes_in_cell_line = pd.unique(pd.concat([group["gene_1"], group["gene_2"]]))
    total_genes = len(genes_in_cell_line)
    
    # 统计能找到 gfm_emb 的基因数
    found_genes = [gene for gene in genes_in_cell_line if gene in gfm_data]
    coverage = len(found_genes) / total_genes * 100  # 百分比
    
    # 统计有效基因对（两个基因都有 emb）
    valid_pairs = group.dropna(subset=["gene_1_emb", "gene_2_emb"])
    valid_ratio = len(valid_pairs) / len(group) * 100
    
    # 记录结果
    results.append({
        "cell_line_origin": cell_line,
        "total_genes": total_genes,
        "found_genes": len(found_genes),
        "coverage (%)": coverage,
        "total_pairs": len(group),
        "valid_pairs": len(valid_pairs),
        "valid_ratio (%)": valid_ratio,
    })

# 转换为 DataFrame
coverage_df = pd.DataFrame(results)

# 按覆盖率排序
coverage_df = coverage_df.sort_values("coverage (%)", ascending=False)

print(coverage_df)

   cell_line_origin  total_genes  found_genes  coverage (%)  total_pairs  \
10             HT29           25           25    100.000000          300   
2              786O           25           25    100.000000          300   
0             22RV1           50           48     96.000000         1225   
17           OVCAR8           75           71     94.666667         1525   
1              293T           73           69     94.520548         2628   
12           JURKAT          388          361     93.041237        75078   
13             K562          645          565     87.596899       120919   
21             RPE1         1856         1564     84.267241        11911   
3              A375         1695         1422     83.893805         1491   
16             MEWO         4435         3329     75.062007         6050   
4              A549         3273         2375     72.563397         9062   
15          MELJUSO         3223         2328     72.230841         5159   
8           

In [ ]:
# geneformer emb 在正样本的覆盖率

sl_pairs = sl_data[sl_data["SL_or_not"] == "SL"]

results = []

for cell_line, group in sl_pairs.groupby("cell_line_origin"):
    valid_sl_pairs = group.dropna(subset=["gene_1_emb", "gene_2_emb"])
    coverage = len(valid_sl_pairs) / len(group) * 100
    results.append({
        "cell_line_origin": cell_line,
        "total_SL_pairs": len(group),
        "valid_SL_pairs": len(valid_sl_pairs),
        "SL_coverage (%)": coverage,
    })
sl_coverage_df = pd.DataFrame(results)

sl_coverage_df = sl_coverage_df.sort_values("SL_coverage (%)", ascending=False)
print(sl_coverage_df)

   cell_line_origin  total_SL_pairs  valid_SL_pairs  SL_coverage (%)
9              HT29               3               3       100.000000
1              293T              59              57        96.610169
2              A375              46              44        95.652174
5              HELA             149             139        93.288591
16           OVCAR8              56              52        92.857143
11           JURKAT             374             342        91.443850
0             22RV1             274             241        87.956204
12             K562            1563            1373        87.843890
18              PC9              87              76        87.356322
20             RPE1            6833            5414        79.233133
19              PK1             668             429        64.221557
3              A549             682             436        63.929619
6            HS936T             636             403        63.364780
13           MEL202             68